## NYISO Load Prediction - Gaussian Kernel
- Objective: Utilize the NBEATS model to predict NYISO load data using historical data from 2013-01-01 to 2023-12-30.
- Training Scheme
    - Random Sampling: For each sample, randomly select a date and use the 24 time steps (hours) as test data. The preceding 200 days are used as training data.
    - Repetition: Sample 100 unique pairs of training and test datasets.
    - Scaling and Prediction: Apply scaling methods on the training data, train the NBEATS model, and then use it to generate predictions for the test data.
- Scaling methods: [definition](https://nixtlaverse.nixtla.io/neuralforecast/common.scalers.html)
     - [`identity`](https://nixtlaverse.nixtla.io/neuralforecast/common.scalers.html#std-statistics)
     - `revin`:  learnable normalization parameters are added on top of the usual normalization technique.
     - `smoothing`: Apply Gaussian kernel with $\sigma =  448$

In [1]:
import os
import io
import pickle
import logging
import warnings
from datetime import timedelta

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
from scipy.ndimage import gaussian_filter1d
from IPython.display import display, clear_output

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, Select, CustomJS, DatetimeTickFormatter
from bokeh.layouts import column
from bokeh.palettes import Category10

import ipywidgets as widgets
from ipywidgets import HBox, VBox
from ipywidgets.embed import embed_minimal_html
import pandas as pd
import numpy as np
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display, clear_output

# Suppress warnings
warnings.filterwarnings("ignore")

In [2]:
class Normalizer:
    def __init__(self, window_size=7, sigma=2):
        """
        Initialize the normalizer with kernel smoothing parameters.
        
        Parameters:
        - window_size: The size of the window for rolling standard deviation.
        - sigma: The standard deviation for Gaussian kernel smoothing.
        """
        self.window_size = window_size
        self.sigma = sigma
        self.smoothed_y = None
        self.std_y = None

    def fit(self, y):
        # Fill NaN values before fitting
        y = pd.Series(y).fillna(method='ffill').fillna(method='bfill').values
        
        # Apply Gaussian kernel smoothing
        self.smoothed_y = gaussian_filter1d(y, sigma=self.sigma)
        
        # Calculate rolling standard deviation
        self.std_y = pd.Series(y).rolling(window=self.window_size, min_periods=1).std().fillna(method='bfill').values
        
        return self

    def transform(self, y):
        # Fill NaN values before transforming
        y = pd.Series(y).fillna(method='ffill').fillna(method='bfill').values
        
        if self.smoothed_y is None or self.std_y is None:
            raise RuntimeError("The normalizer must be fitted before calling transform.")
        
        smoothed_y_partial = self.smoothed_y[-len(y):]
        std_y_partial = self.std_y[-len(y):]

        # Handle any NaN or zero values in std_y_partial to avoid extreme values
        std_y_partial = np.where(np.isnan(std_y_partial) | (std_y_partial == 0), 1e-6, std_y_partial)
        
        normalized_y = (y - smoothed_y_partial) / std_y_partial

        # Forward-fill any remaining NaN values in normalized_y
        normalized_y = pd.Series(normalized_y).fillna(method='ffill').values
        
        return normalized_y

    def fit_transform(self, y):
        self.fit(y)
        return self.transform(y)

    def inverse_transform(self, normalized_y):
        if self.smoothed_y is None or self.std_y is None:
            raise RuntimeError("The normalizer must be fitted before calling inverse_transform.")
        
        smoothed_y_partial = self.smoothed_y[-len(normalized_y):]
        std_y_partial = self.std_y[-len(normalized_y):]
        
        denormalized_y = normalized_y * std_y_partial + smoothed_y_partial
        return denormalized_y


def calculate_overall_mse(zone_dfs, true_col='y', prediction_cols=['NBEATS - Identity', 'NBEATS - Reinv', 'NBEATS - Smoothing']):
    mse_results = {col: [] for col in prediction_cols}
    traning_len = len(zone_dfs)

    # Loop through all the DataFrames in the list
    for df in zone_dfs:
        # Ensure the DataFrame is properly formatted and named
        df = df.rename(columns={'NBEATS1': 'NBEATS - Reinv'})
        
        # Drop rows with NaN values in any of the relevant columns
        df = df.dropna(subset=[true_col] + prediction_cols)
        
        y_true = df[true_col].iloc[-24:]
        
        for col in prediction_cols:
            mse = mean_squared_error(y_true, df[col].iloc[-24:])
            mse_results[col].append(mse)
    
    #return {col: (np.mean(mse_results[col]), np.std(mse_results[col])/np.sqrt(traning_len)) for col in prediction_cols}
    return {col: round(np.mean(mse_results[col]), 3) for col in prediction_cols}



def calculate_mse(y_true, y_pred):
    return mean_squared_error(y_true[-24:], y_pred[-24:])

def plot_prediction(plot_df, zone):
    # Rename the column for consistency
    plot_df = plot_df.rename(columns={
        'NBEATS1': 'NBEATS - Reinv',  # Corrected to 'Reinv' to match the previous context
    })
    
    # Reset index to make the timestamp a column
    plot_df = plot_df.reset_index()
    
    # Calculate MSE for the last 24 values
    y_true = plot_df['y']
    mse_identity = calculate_mse(y_true, plot_df['NBEATS - Identity'])
    mse_reinv = calculate_mse(y_true, plot_df['NBEATS - Reinv'])
    mse_smoothing = calculate_mse(y_true, plot_df['NBEATS - Smoothing'])

    # Create the plot
    p = figure(title=f"NYISO - {zone}", x_axis_type='datetime', x_axis_label='Timestamp [t]', y_axis_label='Load', width=1200, height=600)
    
    # Define colors for the plot lines
    colors = Category10[len(plot_df.columns)]
    
    # Add the true value line
    p.line(x='ds', y='y', source=ColumnDataSource(plot_df), line_width=2, color='black', legend_label='True Value')

    # Add the prediction lines with MSE in the legend
    p.line(x='ds', y='NBEATS - Identity', source=ColumnDataSource(plot_df), line_width=2, color=colors[0], legend_label=f"NBEATS - Identity (MSE: {mse_identity:.2f})")
    p.line(x='ds', y='NBEATS - Reinv', source=ColumnDataSource(plot_df), line_width=2, color=colors[1], legend_label=f"NBEATS - Reinv (MSE: {mse_reinv:.2f})")
    p.line(x='ds', y='NBEATS - Smoothing', source=ColumnDataSource(plot_df), line_width=2, color=colors[2], legend_label=f"NBEATS - Smoothing (MSE: {mse_smoothing:.2f})")
    
    # Legend formatting
    p.legend.title = ''
    p.legend.title_text_font_size = '12pt'
    p.legend.label_text_font_size = '10pt'
    p.legend.location = 'top_left'
    
    # X-axis date formatting
    p.xaxis.formatter = DatetimeTickFormatter(
        days="%Y-%m-%d",
        months="%Y-%m-%d",
        years="%Y-%m-%d"
    )
    
    return p

### Overall MSE

In [3]:
with open('all_prediction_dfs.pkl', 'rb') as f:
    all_prediction_dfs = pickle.load(f)
    
results = pd.DataFrame({zone: calculate_overall_mse(zone_dfs) for zone, zone_dfs in all_prediction_dfs.items()})

results

,CAPITL,CENTRL,DUNWOD,GENESE,HUD VL,LONGIL,MHK VL,MILLWD,N.Y.C.,NORTH,WEST
NBEATS - Identity,9578.937,10209.877,3319.508,6754.199,12800.643,58758.507,2870.185,1202.236,142179.961,463.333,7099.940
NBEATS - Reinv,9626.694,10937.388,3353.237,8275.491,11693.383,66538.396,3108.115,1085.522,141201.899,591.011,8675.762
NBEATS - Smoothing,10676.356,11338.941,3482.413,6261.970,12377.125,68452.575,3203.155,1249.868,146952.140,512.204,8782.506


### Residual Boxplot

In [12]:
def prepare_data(zone):
    # Filter out empty DataFrames
    valid_dfs = [df for df in all_prediction_dfs[zone] if not df.empty]
    
    if not valid_dfs:  # If no valid DataFrames, return an empty DataFrame
        return pd.DataFrame()
    
    # Concatenate all valid DataFrames in the list for the given zone
    df = pd.concat(valid_dfs, ignore_index=True)
    
    # Rename 'NBEATS1' to 'NBEATS - Reinv'
    df = df.rename(columns={'NBEATS1': 'NBEATS - Reinv'})
    
    # Calculate residuals for each prediction model
    residuals = {}
    prediction_cols = ['NBEATS - Identity', 'NBEATS - Reinv', 'NBEATS - Smoothing']
    
    for col in prediction_cols:
        if col in df.columns:
            residuals[col] = df['y'] - df[col]
        else:
            residuals[col] = pd.Series([np.nan] * len(df))
    
    # Convert the residuals dictionary to a DataFrame and melt it for plotting
    residuals_df = pd.DataFrame(residuals)
    residuals_df = residuals_df.melt(var_name='Model', value_name='Residuals')
    
    return residuals_df.dropna()

# Prepare data for all zones and handle empty data
zones = list(all_prediction_dfs.keys())
zone_data = {zone: prepare_data(zone) for zone in zones if not prepare_data(zone).empty}

# Check if we have any valid data after filtering
if not zone_data:
    raise ValueError("No valid data available for any zone.")

# Update the zones list to only include those with valid data
zones = list(zone_data.keys())

# Function to create the plot using Plotly
def create_plot(selected_zone):
    fig = px.box(zone_data[selected_zone], x='Model', y='Residuals',
                 title=f'Residuals Distribution for {selected_zone}')
    return fig

# Function to update the plot based on dropdown selection
def update_plot(change):
    selected_zone = dropdown_zone.value
    with output_widget:
        clear_output(wait=True)
        fig = create_plot(selected_zone)
        fig.show()

# Dropdown widget for zone selection
dropdown_zone = widgets.Dropdown(
    options=zones,
    value=zones[0],
    description='Select Zone:',
    style={'description_width': 'initial'}
)

# Output widget to display the plot
output_widget = widgets.Output()

# Observe changes in the dropdown
dropdown_zone.observe(update_plot, names='value')

# Initial display
display(dropdown_zone, output_widget)

# Display the initial plot
with output_widget:
    fig = create_plot(zones[0])
    fig.show()


Dropdown(description='Select Zone:', options=('CAPITL', 'CENTRL', 'DUNWOD', 'GENESE', 'HUD VL', 'LONGIL', 'MHK…

Output()

### Prediction

In [5]:
output_notebook()
zones = list(all_prediction_dfs.keys())
initial_zone = zones[0]
initial_sample_index = 0

output_prediction = widgets.Output()

def update_plot(change=None):
    selected_zone = dropdown_zone.value
    selected_sample_index = dropdown_sample_index.value
    
    # Update prediction plot
    with output_prediction:
        clear_output(wait=True)
        plot_df = all_prediction_dfs[selected_zone][selected_sample_index].drop("ds", axis=1).iloc[-168:].drop("unique_id", axis=1)
        fig_prediction = plot_prediction(plot_df, selected_zone)
        show(fig_prediction, notebook_handle=True)

# Dropdown for selecting zone
dropdown_zone = widgets.Dropdown(
    options=zones,
    value=initial_zone,
    description='Select Zone:',
    style={'description_width': 'initial'}
)
dropdown_zone.observe(update_plot, names='value')

# Dropdown for selecting sample index
sample_indices = list(range(len(all_prediction_dfs[initial_zone])))
dropdown_sample_index = widgets.Dropdown(
    options=sample_indices,
    value=initial_sample_index,
    description='Select Sample Index:',
    style={'description_width': 'initial'}
)
dropdown_sample_index.observe(update_plot, names='value')

layout = widgets.VBox([dropdown_zone, dropdown_sample_index, output_prediction])

display(layout)
update_plot()

Loading BokehJS ...